# GPU Testing Notebook

Small notebook to do a quick 'sniff test' of CUDA/GPU capabilities for Jetstream2 VM's. This is not an exhaustive test! Whole notebook should run in less than a minute, and might return some errors, but every cell should run.   

In [ ]:
!nvidia-smi

In a terminal window, if you want to watch/monitor this information, use the below code. This will refresh the output every second. 

In [ ]:
# watch -n 1 nvidia-smi

## Tensorflow Check

In [ ]:
import tensorflow as tf

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
is_cuda_gpu_available = tf.config.list_physical_devices('GPU')
is_cuda_gpu_available

## PyTorch Check

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.device(0)

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')

In [ ]:
torch.cuda.get_device_properties(0)

In [ ]:
t = torch.randn(9,9)
t.is_cuda  # returns False

In [ ]:
t = torch.randn(2,2).cuda()
t.is_cuda  # returns True

In [ ]:
t = t.cpu()
t.is_cuda  # returns False

## XGBoost Check

In [ ]:
import xgboost as xgb

# Dataset we will load in
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()
X = iris["data"]
y = iris["target"]
xgb_model = xgb.XGBRegressor(
        # If there is no GPU, the tree_method kwarg will cause either
        # - an error in `xgb_model.fit(X, y)` (seen with pytest) or
        # - a warning printed to the console (seen in Spyder)
        # It's unclear which of the two happens under what circumstances.
        tree_method="gpu_hist" # GPU!!
    )
xgb_model.fit(X, y)
# Check that no warning was printed.

## Catboost Check

In [ ]:
import catboost

from catboost.utils import get_gpu_device_count
from catboost import CatBoostRegressor
catboost.__version__

In [ ]:
print('I see %i GPU devices' % get_gpu_device_count())

    I see 1 GPU devices


Below seems like a catboost issue, not a GPU one

In [ ]:
model = CatBoostRegressor(iterations=1000,
                          task_type="GPU",
                          gpu_ram_part=0.5
                         # n_models = 0.05
                         # devices='1'
                         )
model.fit(X,
          y,
          verbose=False)

## CuPy

In [ ]:
import cupy as cp
x = cp.arange(6).reshape(2, 3).astype('f')
x

In [ ]:
x.sum(axis=1)